# Zeitnachweise auswerten
### PDF-Dateien zu Datensatz zusammenfassen

In [ ]:
Ordner_für_Zeitnachweise = '/Users/ozguncakir/Repositories/PDF_Extrahieren/Zeitnachweise'

## Libraries einlesen

In [ ]:
import pandas as pd
import PyPDF2
import numpy as np
from datetime import datetime
import plotly.express as px
import tabula
import re
import os
import fnmatch
import pkg_resources

### Kontrolliere ob eine ausreichende Pandas Version installiert ist

In [ ]:
installed_packages_list = sorted(["%s==%s" % (i.key, i.version) for i in pkg_resources.working_set])
matching_strings = float(fnmatch.filter(installed_packages_list, "pandas==*")[0].strip('pandas==')[:3])

if matching_strings<1.3:
    print('Du brauchst mindestens Pandas 1.3.0')

## Funktionen definieren

In [ ]:
def get_float_sign(column, df):
    df[column] = np.where(
        [str(d).__contains__('-') for d in df[column]],
        [('-'+str(d).replace('-','').replace(',','.').replace(' ','')) for d in df[column]],
        [(str(d).replace(',','.').replace(' ','')) for d in df[column]]).astype(float)
    return df[column]

In [ ]:
def column_to_float(columns_list, df):
    for column in columns_list:
        df[column] = np.where(
            [str(d).__contains__('-') for d in df[column]],
            [('-'+str(d).replace('-','').replace(',','.')) for d in df[column]],
            [(str(d).replace(',','.')) for d in df[column]]
            ).astype(float)
    return df[column]

In [ ]:
def strip_date(columns_list, df):
    for column in columns_list:
        df[column]=df[column].dt.time

In [ ]:
def get_date(df, data, column, search, page, occurence, start, end):
    df[column] = pd.to_datetime(data[page].split(search)[occurence][start:end], dayfirst=True)
    df[column] = df[column].dt.date
    return df[column]

## Daten bearbeiten

### Aggregierte Daten

In [ ]:
dfs = []
result = pd.DataFrame()
for file_name in os.listdir(Ordner_für_Zeitnachweise):
    file_path = os.path.join(Ordner_für_Zeitnachweise, file_name)
    if os.path.isfile(file_path):
        _, file_extension = os.path.splitext(file_path)
        df_list = []
        if file_extension == ".pdf":
            reader = PyPDF2.PdfReader(file_path)
            num_pages = len(reader.pages) 
        data = []
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            data.append(page.extract_text())  
        meta = reader.metadata
        meta_dict = dict(meta)
        index = range(len(meta_dict))
        df = pd.DataFrame(meta_dict, index=index).drop_duplicates()
        df['Zeitraum von'] = get_date(df, data, 'Zeitraum von', 'Zeitraum', 0, 1, 2, 12)
        df['Zeitraum bis'] = get_date(df, data, 'Zeitraum von', 'Zeitraum', 0, 1, 15, 25)
        df['Personalnummer'] = data[0].split("Personalnummer")[1][2:10]
        df['Personalbereich'] = data[0].split("Personalbereich")[1][2:7]
        df['Teilbereich'] = data[0].split("Teilbereich")[1][2:6]
        df['Mitarbeiterkreis'] = data[0].split("Mitarbeiterkreis")[1][2:20]
        df['Kostenstelle'] = data[0].split("Kostenstelle")[1][2:12]
        df['Org.-Einheit'] = data[0].split("Org.-Einheit")[1][2:8]
        df['Arbeitszeitplan'] = data[0].split("Arbeitszeitplan")[1][2:7]
        df['Anteil %'] = float(data[0].split("Anteil %")[1][2:7].replace(',','.'))
        df['Status'] = data[0].split("Status")[1][2:20]
        df['Anzahl Seiten'] = len(reader.pages)
        df['Monatsübersicht zum Stichtag'] = data[1].split("Monatsübersicht zum Stichtag")[1][1:11]
        df['anrechenbare Istzeit'] = float(data[1].split("anrechenbare Istzeit")[1][3:9].replace(',','.'))
        df['Sollzeit'] = float(data[1].split("Sollzeit")[1][3:9].replace(',','.'))
        df['Zeit-Saldo akt.Periode'] = data[1].split("Zeit-Saldo akt.Periode")[1][3:10]
        df['Zeit-Saldo Vorperiode'] = data[1].split("Zeit-Saldo Vorperiode")[1][3:10]
        df['Zeit-Saldo zur Auszahlung'] = float(data[1].split("Zeit-Saldo zur Auszahlung")[1][3:10].replace(',','.'))
        df['Zeit-Saldo gesamt'] = data[1].split("Zeit-Saldo gesamt")[1][3:10]
        df['Überzeiten'] = float(data[1].split("Überzeiten")[1][3:9].replace(',','.'))
        df['Überzeit (Tag>10 Std.)'] = float(data[1].split("Überzeit (Tag>10 Std.)")[1][3:9].replace(',','.'))
        df['Überzeit außerhalb der Rahmenzeit'] = float(data[1].split("Überzeit außerhalb der Rahmenzeit")[1][3:9].replace(',','.'))
        df['Mehrarbeit Grundstunden'] = float(data[1].split("Mehrarbeit Grundstunden")[1][3:9].replace(',','.'))
        df['Mehrarbeit Zuschlag normal'] = float(data[1].split("Mehrarbeit Zuschlag normal")[1][3:9].replace(',','.'))
        df['Mehrarbeit Zuschlag Samstag'] = float(data[1].split("Mehrarbeit Zuschlag Samstag")[1][3:9].replace(',','.'))
        df['Mehrarbeit Zuschlag Sonntag'] = float(data[1].split("Mehrarbeit Zuschlag Sonntag")[1][3:9].replace(',','.'))
        df['Mehrarbeit Zuschlag Feiertag'] = float(data[1].split("Mehrarbeit Zuschlag Feiertag")[1][3:9].replace(',','.'))
        df['Nachtzuschlag'] = float(data[1].split("Nachtzuschlag")[1][3:9].replace(',','.'))
        df['Mehrarbeitskonto /Freizeit'] = float(data[1].split("Mehrarbeitskonto /Freizeit")[1][3:9].replace(',','.'))
        df['Resturlaub in Tagen'] = float(data[1].split("Resturlaub")[1][3:9].replace(',','.'))
        df['Zeit-Saldo akt.Periode'] = get_float_sign('Zeit-Saldo akt.Periode', df)
        df['Zeit-Saldo Vorperiode'] = get_float_sign('Zeit-Saldo Vorperiode', df)
        df['Zeit-Saldo gesamt'] = get_float_sign('Zeit-Saldo gesamt', df)
        dfs.append(df)
# df-Liste zu df machen
df = pd.concat(dfs)
# Reset Index und starte mit 1
df_agg = df.reset_index().drop(columns='index')
df_agg = df_agg.drop(columns= [ '/Author', '/CreationDate', '/Creator', '/Producer'])
df_agg.index = df_agg.index + 1
df_agg.head()

## Einzelliste ziehen

In [ ]:
dfs = []
for file_name in os.listdir(Ordner_für_Zeitnachweise):
    file_path = os.path.join(Ordner_für_Zeitnachweise, file_name)
    if os.path.isfile(file_path):
        _, file_extension = os.path.splitext(file_path)
        df_list = []
        if file_extension == ".pdf":
            tables = tabula.read_pdf(file_path, pages = "all", multiple_tables = True)
            # Zwei Seiten reichen für alle Zeitnachweise
            df_1 = pd.DataFrame(tables[1])                  # Tabelle der ersten Seite
            df_1.drop(df_1.index[-1], inplace=True)         # lösche letzte Zeile
            df_2 = pd.DataFrame(tables[2])                  # Tabelle der zweiten Seite
            df_2.drop(df_2.index[-1], inplace=True)         # lösche letzte Zeile
            df_3 = pd.concat([df_1,df_2])                   # Zusammenführung beider Tabellen
            # Zusammenfügen beider Zeilen der Tabellen zu einer Zeile
            result = df_3.iloc[0].astype(str).str.cat(df_3.iloc[1].astype(str), sep='\r')
            df = pd.DataFrame(result).transpose()
            # Umbennenungen
            df = df.rename(columns={
                'Unnamed: 0':'Kommt & Geht',
                'Kommt\rUhrz.     Term.':'Ist-Zeit',
                'Geht\rUhrz.     Term.':'Über-Zeit',
                'Pause':'Sollzeit',
                'Ist-\rZeit':'Pause',
                'Über-\rZeit':'anrech. Istzeit',
                'anrech.\rIstzeit':'Zeit-Saldo',
                'Soll-\rzeit':'Mehrzeit Grundst.',
                'Zeit-\rSaldo':'Zuschlag'
                })
            # entferne leere Spalten
            df = df.drop(columns=['Mehrzeit\rGrundst.', 'Zuschl.'])
            # String zu Liste transformieren
            for col in df.columns:
                df[col].iloc[0] = df[col].str.split("\r").iloc[0]
            # Index aktualisieren
            df.reset_index(inplace=True)
            df.drop(columns=['index'], inplace=True)
            # Meistens müssen die zwei aufeinanderfolgenden Elemenete für Zeitstempelpaare zusammengeführt werden, es sei denn die Pause wurde nicht ausgebucht 
            condition = lambda x: x.startswith(('07', '08', '09', '10'))
            condition_2 = lambda x: x.startswith(('1', '0'))
            my_list = df['Kommt & Geht'].iloc[0]
            new_list = []
            temp = my_list[0]
            for i in range(1, len(my_list)):
                if condition(my_list[i]) or not condition_2(my_list[i]):
                    new_list.append(temp)
                    temp = my_list[i]
                else:
                    temp += my_list[i]
            new_list.append(temp)
            df['Kommt & Geht'].iloc[0]=new_list
            # Bei Feiertagen werden keine zugehörigen Zeilen hinzugefügt, hier füge ich dafür nan Werte ein
            for count, val in enumerate(df['Kommt & Geht'].iloc[0]):
                if not val.startswith(('1', '0')) and not val=='Urlaub':
                    df['Ist-Zeit'].iloc[0].insert(count, np.nan)
                    df['Über-Zeit'].iloc[0].insert(count, np.nan)
                    df['Pause'].iloc[0].insert(count, np.nan)
                    df['anrech. Istzeit'].iloc[0].insert(count, np.nan)
                    df['Sollzeit'].iloc[0].insert(count, np.nan)
                    df['Zeit-Saldo'].iloc[0].insert(count, np.nan)
                    df['Mehrzeit Grundst.'].iloc[0].insert(count, np.nan)
                    df['Zuschlag'].iloc[0].insert(count, np.nan)
                if not val.startswith(('1', '0')):
                    df['Kommt & Geht'].iloc[0][count] = '00:0000:00'
            # Zellen mit Listen in Zeilen exploden (min. pandas 1.3 nötig)
            df = df.explode(column=['Tag', 'Kommt & Geht', 'Ist-Zeit', 'Über-Zeit', 'Pause', 'anrech. Istzeit', 'Sollzeit', 'Zeit-Saldo', 'Mehrzeit Grundst.', 'Zuschlag'])
            # Spalten 'regexen'
            df['Büro'] = np.where(df['Kommt & Geht'].str.contains('H'), 'Büro', 'Home-Office')
            df['Haus'] =  np.where(df['Kommt & Geht'].str.contains('H'), df['Kommt & Geht'].apply(lambda x: x[x.find('H'):x.find('H')+3]), 'Zuhause')
            df['Kommt'] = pd.to_datetime(df['Kommt & Geht'].str.slice(stop=5), format='%H:%M')
            df['Zeit'] = df['Kommt & Geht'].apply(lambda x: re.sub('H.{4}', '1', x) if 'H' in x else x)
            df['Zeit'] = df['Zeit'].apply(lambda x: re.sub('H.{4}', '1', x).replace(x[x.find('H'):x.find('H')+4],''))
            df['Geht'] = pd.to_datetime(df['Zeit'].str.slice(start=-5), format='%H:%M')
            df['Pause-Anfang'] = df['Zeit'].apply(lambda x: x[5:10])
            df['Pause-Anfang'] = pd.to_datetime(np.where( df['Pause-Anfang'] == df['Zeit'].str.slice(start=-5), None, df['Pause-Anfang']), format='%H:%M')
            df['Pause-Ende'] = df['Zeit'].apply(lambda x: x[10:15])
            df['Pause-Ende'] = pd.to_datetime(np.where( df['Pause-Ende'] == '', None, df['Pause-Ende']), format='%H:%M')
            df['Gesamt-Zeit'] = (df['Geht'] - df['Kommt']) / pd.to_timedelta(1, unit='H')
            df['Vormittag-Zeit'] = (df['Pause-Anfang'] - df['Kommt']) / pd.to_timedelta(1, unit='H')
            df['Pausen-Dauer'] = (df['Pause-Ende'] - df['Pause-Anfang']) / pd.to_timedelta(1, unit='H')
            df['Nachmittag-Zeit'] = (df['Geht'] - df['Pause-Ende']) / pd.to_timedelta(1, unit='H')
            df['Datum'] = [datetime(int(tables[3].columns[0][-4:]), int(tables[3].columns[0][-7:-5]), int(d[:2])) for d in df['Tag'] ]
            df['Wochentag'] = df['Tag'].apply(lambda x: x[2:])
            strip_date(['Kommt', 'Geht', 'Pause-Anfang', 'Pause-Ende'], df)
            # Zwischenzeit nach Arbeitstagen, Urlaub und Feiertagen unterscheiden
            df['Arbeitstag'] = np.where( [type(x)==float for x in df['Ist-Zeit']], 'Feiertag', np.where( df['Gesamt-Zeit']==0, 'Urlaub', 'Arbeitstag')) 
            # Zahlen im String formatieren
            column_to_float(['Ist-Zeit', 'Über-Zeit', 'Pause', 'anrech. Istzeit', 'Sollzeit', 'Mehrzeit Grundst.', 'Zuschlag', 'Zeit-Saldo'], df)
            df['Pause-Gesamt'] = df['Pausen-Dauer']+df['Pause']
            # unnötige Spalten droppen
            df = df.drop(columns=['Tag', 'Zeit', 'Kommt & Geht'])
            # df zu df-Liste hinzufügen
            dfs.append(df)
# df-Liste zu df machen
df = pd.concat(dfs)
# Reset Index und starte mit 1
df = df.reset_index().drop(columns='index')
df.index = df.index + 1
df.head()

## Export als Excel-Datei

In [ ]:
dfs = {'Zusammenfassung': df_agg, 'Einzelliste': df} # dict of sheetnames: dataframes

writer = pd.ExcelWriter('Zeitnachweise.xlsx', engine='xlsxwriter')
for sheetname, df in dfs.items():  # loop through dict of dataframes
    (max_row, max_col) = df.shape 
    df.to_excel(writer, sheet_name=sheetname, index=False)  # send df to writer
    workbook  = writer.book
    workbook.formats[0].set_font_name('Source Sans Pro')
    workbook.formats[0].set_border(2)
    worksheet = writer.sheets[sheetname]  # pull worksheet object
    worksheet.autofilter(0, 0, df.shape[0], df.shape[1]-1) # activate filter on column headers
    header_format = workbook.add_format({ # define header format
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#006E9D',
        'font_color': 'white',
        'border': 0,
        'font_name': 'Source Sans Pro'
        })
    for idx, col in enumerate(df):  # loop through all columns
        series = df[col]
        max_len = max(
            series.astype(str).map(len).max(),  # len of largest item
            len(str(series.name))  # len of column name/header
            ) + 5  # adding more space
        worksheet.set_column(idx, idx, max_len)  # set column width
        for col_num, value in enumerate(df.columns.values):
            worksheet.write(0, col_num, value, header_format) # set header format
writer.close()

In [ ]:
# auf Arbeitstage filtern
main_df = df[df['Arbeitstag']=='Arbeitstag'].reset_index().drop(columns='index')
main_df.index = main_df.index + 1

### Arbeitstage, Urlaub und Feiertage

In [ ]:
num_workdays = len(df[df['Arbeitstag']=='Arbeitstag'])
num_vacation = len(df[df['Arbeitstag']=='Urlaub'])
num_holidays = len(df[df['Arbeitstag']=='Feiertag'])
num_weekends = (df['Datum'].max() - df['Datum'].min()).days - len(df)
share_workdays = "{0:.0%}".format(len(df[df['Arbeitstag']=='Arbeitstag']) / (df['Datum'].max() - df['Datum'].min()).days)

fig = px.pie(
    df,
    names=['Arbeitstag', 'Wochenende', 'Urlaub', 'Feiertag'],
    values=[num_workdays, num_weekends, num_vacation, num_holidays],
    color_discrete_sequence=['#0068C9', '#00A0E7', '#00D0E0', '#70FACB'],
    title = 'An '+share_workdays + ' der Tage hast du gearbeitet' + '<br><sup>Unterteilung des Zeitraums in Arbeitstage, Urlaub und Feiertage</sup>')
fig.update_traces(textposition='inside', textinfo='value+label')


### Wo du arbeitest

In [ ]:
haus = df.groupby(by='Haus').size().sort_values(ascending=False)
haus_most = haus[haus.index!='Zuhause'].index[0].replace('H','').replace('0','')
haus_most_days = str(haus[haus.index!='Zuhause'][0])

fig = px.pie(
    df,
    names='Haus',
    color_discrete_sequence=['#0068C9', '#00A0E7', '#00D0E0', '#70FACB'],
    title='Im Büro warst du am häufigsten in Haus ' + haus_most + '<br><sup>In welchem Haus du arbeitest</sup>')
fig.update_traces(textposition='inside', textinfo='value+label')
fig.update_traces(hovertemplate="%{label}: %{value} Tage<extra></extra>")

### Deine Home-Office-Quote

In [ ]:
ho_quota = "{0:.0%}".format(len(main_df[main_df['Büro']=='Home-Office']) / len(main_df))
diff_ho = int(0.4*len(main_df)) - len(main_df[(main_df['Büro']=='Büro')])
if diff_ho>0: 
    delta_ho = str(diff_ho) + ' Tage hättest du häufiger im Büro sein müssen'
else:
    delta_ho = str(diff_ho) + ' Tage hättest du weniger im Büro sein können'

fig = px.pie(
    df,
    names='Büro',
    color_discrete_sequence=['#0068C9', '#00A0E7', '#00D0E0', '#70FACB'],
    title=ho_quota + ' der Tage verbringst du im Home-Office' + '<br><sup>' + delta_ho + '</sup>')
fig.update_traces(textposition='inside', textinfo='value+label')
fig.update_traces(hovertemplate="%{label}: %{value} Tage<extra></extra>")

In [ ]:
ho_df = main_df.groupby(by=['Büro', 'Wochentag']).size().sort_values(ascending=False).reset_index()
ho_df[ho_df['Büro']=='Home-Office']

fig = px.bar(
    ho_df,
    x='Wochentag',
    y=0,
    color='Büro',
    title='Home-Office im Wochenverlauf',
    category_orders={'Wochentag':['MO', 'DI', 'MI', 'DO', 'FR']},
    color_discrete_sequence=['#0068C9', '#00A0E7'],
    labels={'0': "Anzahl Tage"})
fig.update_traces(hovertemplate="%{label}: %{value} Tage<extra></extra>")

### Kombination der oberen Visualisierungen

In [ ]:
fig = px.sunburst(
    df, 
    path=['Arbeitstag', 'Büro', 'Haus'],
    color_discrete_sequence=['#0068C9', '#00A0E7', '#00D0E0', '#70FACB'],
    title = "Anteile der Arbeitstage und wo du arbeitest<br><sup>Klick doch in die Kreise</sup>")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_traces(hovertemplate="%{label}: %{value} Tage<extra></extra>")